In [1]:
import cv2
import time
import math
import asyncio
import pyautogui
import numpy as np

from mss import mss
from enum import Enum

In [2]:
%run Pathfinding.ipynb
%run astar.ipynb

# Custom Classes

In [3]:
class Direction(Enum):
    TOP = 1
    LEFT = 2
    BOTTOM = 3
    RIGHT = 4

#  Helper functions

In [4]:
def countdown():
    time.sleep(1)
    print('3')
    time.sleep(1)
    print('2')
    time.sleep(1)
    print('1')

In [5]:
def screenshot():
    field_width = 688
    field_height = 608
    field_left = 607
    field_top = 302
    
    with mss() as sct:        
        monitor_number = 1
        mon = sct.monitors[monitor_number]
        monitor = {'mon': monitor_number, 'width': field_width, 'height': field_height, 'left': mon["left"] + field_left, 'top': mon["top"] + field_top}

        sct_img = sct.grab(monitor)
        img = np.array(sct_img.pixels).astype(np.uint8)[::,::,::-1]
        
        return img

In [6]:
# detect if the game is over yet
def game_over(img):
    (left, top, right, bottom) = (453, 436, 533, 515)
    img_crp = img[top:bottom, left:right]
    
    img_score_path = 'img_raw/gameover.png'
    img_score_full = cv2.imread(img_score_path)
    img_score = img_score_full[top:bottom, left:right]
    
    hist_crp = calc_hist(img_crp)
    hist_score = calc_hist(img_score)
    
    sum_diff = 0
    for chan in range(3):
        diff = cv2.compareHist(hist_crp[chan], hist_score[chan], cv2.HISTCMP_CORREL)
        sum_diff += diff
    
    avg_diff = sum_diff/3
    if(avg_diff > 0.7):
        return True
    
    return False

In [7]:
def calc_direction(current_pos, old_pos):
#     start_time = time.perf_counter()
    
    current_pos_no = 10 * current_pos.x + current_pos.y
    old_pos_no = 10 * old_pos.x + old_pos.y
    pos_diff = current_pos_no - old_pos_no
    
    ret_direction = None
    if(pos_diff == -1):
        ret_direction = Direction.TOP
    elif(pos_diff == -10):
        ret_direction = Direction.LEFT
    elif(pos_diff == 1):
        ret_direction = Direction.BOTTOM
    elif(pos_diff == 10):
        ret_direction = Direction.RIGHT
    
#     duration = time.perf_counter() - start_time
    
    return ret_direction

In [8]:
def press_keyboard_button(direction):
    if(direction == Direction.TOP):
        # press up arrow
        pyautogui.press('up')
        print('up')
    if(direction == Direction.LEFT):
        # press left arrow
        pyautogui.press('left')
        print('left')
    if(direction == Direction.BOTTOM):
        # press down arrow
        pyautogui.press('down')
        print('down')
    if(direction == Direction.RIGHT):
        # press right arrow
        pyautogui.press('right')
        print('right')

In [9]:
def augment_snake_body(cells, head_pos, direction, n_augment):
    if(direction == Direction.TOP):
        for i in range(n_augment):
            cells[head_pos.y - i, head_pos.x] = Label.BODY.value
        new_head_pos = Position(head_pos.y - n_augment, head_pos.x)
        
    if(direction == Direction.BOTTOM):
        for i in range(n_augment):
            cells[head_pos.y + i, head_pos.x] = Label.BODY.value
        new_head_pos = Position(head_pos.y + n_augment, head_pos.x)
            
    if(direction == Direction.LEFT):
        for i in range(n_augment):
            cells[head_pos.y, head_pos.x - i] = Label.BODY.value
        new_head_pos = Position(head_pos.y, head_pos.x - n_augment)
            
    if(direction == Direction.RIGHT):
        for i in range(n_augment):
            cells[head_pos.y, head_pos.x + i] = Label.BODY.value
        new_head_pos = Position(head_pos.y, head_pos.x + n_augment)
        
    cells[new_head_pos.y, new_head_pos.x] = Label.HEAD.value
            
    return cells, new_head_pos

# Snake pathing & controlling functions

In [16]:
# find the optimal path and control the snake to move along the path
async def pathfinding(cells, head_pos, apple_pos, snake_speed, snake_turn_speed):    
    if(head_pos is None):
        print('Null head_pos')
        return None, None, False
    
    cm_pos = center_of_mass(cells, nrows, ncols)
#     min_body_pos = find_min_max_body_pos(cells, nrows, ncols, True)
#     max_body_pos = find_min_max_body_pos(cells, nrows, ncols, False)

    new_cells, path = find_optimal_path(cells, head_pos, [cm_pos], [1])
    if(new_cells is None):
        print('Null new_cells')
        return None, None, False
    
    print(new_cells)        
    print("------------------------")
    
    task = asyncio.create_task(control_snake(new_cells, path, head_pos, apple_pos, snake_speed, snake_turn_speed))
    await task
    
    return task.result()[0], task.result()[1], task.done()

In [18]:
async def control_snake(new_cells, path, head_pos, apple_pos, snake_speed, snake_turn_speed):
    init_direction = None    
    current_direction = init_direction
    old_pos = path[0]
    last_fn_exec_time = 0.06
    
    for i, pos in enumerate(path[1:]):
        direction = calc_direction(pos, old_pos)    # snake's turn direction
        old_pos = pos
        
        if(not direction is None):
            press_keyboard_button(direction)
            
            if(i==len(path)-2):
                await asyncio.sleep(snake_speed - last_fn_exec_time)
                break
            if(current_direction == direction):
                await asyncio.sleep(snake_speed)
            else:
                await asyncio.sleep(snake_turn_speed)
                
            current_direction = direction
    
    near_wall, new_direction = await turn_if_near_wall(direction, apple_pos, snake_turn_speed)
    print('near_wall_direction:', new_direction)
    
    return near_wall, new_direction

In [12]:
async def turn_if_near_wall(direction, head_pos, snake_turn_speed):
    left_wall_x = 0
    right_wall_x = 16
    top_wall_y = 0
    bottom_wall_y = 14
    
    dx1 = head_pos.x - left_wall_x
    dx2 = right_wall_x - head_pos.x
    dy1 = head_pos.y - top_wall_y
    dy2 = bottom_wall_y - head_pos.y
    
    delta_dx = dx1 - dx2
    delta_dy = dy1 - dy2
    
    k = 5    # space from head to wall threshold
    
    ret_direction = None
    if((direction == Direction.LEFT and dx1 <= k) or
       (direction == Direction.RIGHT and dx2 <= k)):
        if(delta_dy > 0):          # dy1 > dy2
            ret_direction = Direction.TOP
        else:
            ret_direction = Direction.BOTTOM
        
    if((direction == Direction.TOP and dy1 <= k) or
       (direction == Direction.BOTTOM and dy2 <= k)):
        if(delta_dx > 0):          # dx1 > dx2
            ret_direction = Direction.LEFT
        else:
            ret_direction = Direction.RIGHT
            
    if(ret_direction is None):
        return False, direction
    else:
        press_keyboard_button(ret_direction)
        await asyncio.sleep(snake_turn_speed)
        return True, ret_direction

#  Main function

In [21]:
async def main():    
    pyautogui.PAUSE = 0.0
    pyautogui.FAILSAFE = False
    threshold = 0
    snake_speed = 0.1713
    snake_turn_speed = 0.17125
    offset_time = 0
    
    first_move = True
    cached_apple_pos = None
    cached_direction = Direction.RIGHT
    path_completed = False
    
    countdown()
    
    while(True):
        start_time = time.time()
        img = screenshot()
        if(game_over(img)):
            print('game over')
            break
            
        cells, head_pos, apple_pos = get_cells(img, Label, threshold)
        
        print(cells)
        print("=============================")        
            
        # Execute if the snake has not eaten the apple yet but finished moving along the path
        if(apple_pos is not None and path_completed and cached_apple_pos == apple_pos):
            exec_time = time.time() - start_time + offset_time
            n_augment = exec_time / snake_speed
            print(n_augment, exec_time)
            n_augment = round(n_augment)
            print(n_augment)
            cells, head_pos = augment_snake_body(cells, head_pos, cached_direction, n_augment)
            
            near_wall, new_direction, path_completed = await pathfinding(cells, head_pos, apple_pos, snake_speed, snake_turn_speed)
            cached_apple_pos = apple_pos
            
            if(new_direction is not None):
                cached_direction = new_direction
                
        # Execute if the apple position has changed
        elif(apple_pos is not None and (cached_apple_pos is None or cached_apple_pos != apple_pos)):
            if(first_move):
                first_move = False
            else:
                exec_time = time.time() - start_time + offset_time
                n_augment = exec_time / snake_speed
                print(n_augment, exec_time)
                n_augment = round(n_augment)
                print(n_augment)
                cells, head_pos = augment_snake_body(cells, head_pos, cached_direction, n_augment)
            
            near_wall, new_direction, path_completed = await pathfinding(cells, head_pos, apple_pos, snake_speed, snake_turn_speed)
            cached_apple_pos = apple_pos
            
            if(new_direction is not None):
                cached_direction = new_direction

In [25]:
await main()

3
2
1
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 2 3 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  2  3 -1 -1 -1 -1 -1 -1 -1 -1  1  0  0  0  0]
 [ 0  

down
down
down
down
left
near_wall_direction: Direction.LEFT
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 2 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 3 2 2 2 0]
 [0 0 0 0 0 0

right
right
right
right
near_wall_direction: Direction.RIGHT
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 2 0 0 0 2 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 2 2 2 2 2 2 2 2 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
game over
